# DATA SPLIT - testing the full approach - COGNITION
saving models in a different folder using pickle, saving them in the dataframe causes xgboost to crash

**DO NOT RUN AGAIN**

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION_v2"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## define a fixed samplerate

In [3]:
samplerate=200

## load cognition

In [4]:
annotations, signalsMetadata = loadCOGNITIONSpindles_v2(cognipath)

In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)

In [6]:
annotations.head()

,subjectId,channel,startTime,stopTime,duration,phase,type,samplerate,startInd,stopInd
0,COG001,F3,1798.073930,1798.696491,0.622561,2,spindle,200,359615,359739
1,COG001,F3,1868.599222,1869.591433,0.992211,2,spindle,200,373720,373918
2,COG001,F3,1914.163425,1915.369645,1.206220,2,spindle,200,382833,383074
3,COG001,F3,1916.653694,1917.840459,1.186765,2,spindle,200,383331,383568
4,COG001,F3,1928.638133,1929.902718,1.264585,2,spindle,200,385728,385981


In [7]:
signalsMetadata.head()

,filename,subjectId,channel,duration,samplerate,isOriginalSamplerate,database
0,COG001_F3.pkl,COG001,F3,39600.0,200,False,COGNITION_v2
1,COG001_C3.pkl,COG001,C3,39600.0,200,False,COGNITION_v2
2,COG001_F4.pkl,COG001,F4,39600.0,200,False,COGNITION_v2
3,COG001_C4.pkl,COG001,C4,39600.0,200,False,COGNITION_v2
4,COG002_F3.pkl,COG002,F3,39600.0,200,False,COGNITION_v2


## create data split
LOOCV with 3 subjects for validation by fold

In [13]:
#discard subject 0001 and 0009
usedSubjects=np.array(signalsMetadata['subjectId'])
#usedSubjects=np.setdiff1d(usedSubjects,['0001','0009'])
usedSubjects=np.unique(usedSubjects)[0:9]
usedSubjects

array(['COG001', 'COG002', 'COG003', 'COG004', 'COG005', 'COG006',
       'COG007', 'COG008', 'COG009'], dtype=object)

In [14]:
valCount=2  #number of signals for validation (selected randomly)

trainSplits=[]
valSplits=[]
testSplits=[]
for i in range(len(usedSubjects)):
    thisTest=usedSubjects[i]
    thisNoTest=np.setdiff1d(usedSubjects,thisTest)
    thisVal=np.random.choice(thisNoTest,size=valCount,replace=False)
    thisTrain=np.setdiff1d(thisNoTest,thisVal)
    trainSplits.append(thisTrain)
    valSplits.append(thisVal)
    testSplits.append(thisTest)

dataSplits=pd.DataFrame({
    'train':trainSplits,
    'val':valSplits,
    'test':testSplits
})

dataSplits

,train,val,test
0,"[COG002, COG003, COG005, COG006, COG007, COG009]","[COG004, COG008]",COG001
1,"[COG003, COG004, COG006, COG007, COG008, COG009]","[COG001, COG005]",COG002
2,"[COG001, COG004, COG006, COG007, COG008, COG009]","[COG002, COG005]",COG003
3,"[COG001, COG003, COG005, COG006, COG007, COG008]","[COG002, COG009]",COG004
4,"[COG003, COG004, COG006, COG007, COG008, COG009]","[COG002, COG001]",COG005
5,"[COG002, COG003, COG004, COG005, COG008, COG009]","[COG001, COG007]",COG006
6,"[COG001, COG002, COG003, COG004, COG005, COG009]","[COG008, COG006]",COG007
7,"[COG001, COG002, COG004, COG005, COG007, COG009]","[COG006, COG003]",COG008
8,"[COG001, COG002, COG003, COG004, COG006, COG008]","[COG007, COG005]",COG009


In [15]:
dumpPickle("dataSplits_LOOCV.pkl",dataSplits)